In [1]:
import maite

In [16]:
from maite import list_datasets

list_datasets(provider="torchvision")[:20]

['CIFAR10',
 'CIFAR100',
 'CLEVRClassification',
 'CREStereo',
 'Caltech101',
 'Caltech256',
 'CarlaStereo',
 'CelebA',
 'Cityscapes',
 'CocoCaptions',
 'CocoDetection',
 'Country211',
 'DTD',
 'DatasetFolder',
 'EMNIST',
 'ETH3DStereo',
 'EuroSAT',
 'FER2013',
 'FGVCAircraft',
 'FakeData']

In [3]:
list_datasets(provider="huggingface")

/home/jtsexton/.local/lib/python3.10/site-packages/huggingface_hub/utils/endpoint_helpers.py:162: FutureWarning: 'DatasetFilter' is deprecated and will be removed in huggingface_hub>=0.24. Please pass the filter parameters as keyword arguments directly to the `list_datasets` method.
  warnings.warn(


[DatasetInfo(id='acronym_identification', author=None, sha='15ef643450d589d5883e289ffadeb03563e80a9e', created_at=datetime.datetime(2022, 3, 2, 23, 29, 22, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 1, 9, 11, 39, 57, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=793, likes=18, paperswithcode_id='acronym-identification', tags=['task_categories:token-classification', 'annotations_creators:expert-generated', 'language_creators:found', 'multilinguality:monolingual', 'size_categories:10K<n<100K', 'source_datasets:original', 'language:en', 'license:mit', 'acronym-identification', 'croissant', 'arxiv:2010.14678', 'region:us'], card_data=None, siblings=None),
 DatasetInfo(id='ade_corpus_v2', author=None, sha='4ba01c71687dd7c996597042449448ea312126cf', created_at=datetime.datetime(2022, 3, 2, 23, 29, 22, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 1, 9, 11, 42, 58, tzinfo=datetime.timezone.utc), private=Fa

In [5]:
from maite import load_dataset
dataset = load_dataset(
    provider="huggingface",
    dataset_name="cifar10",
    task="image-classification",
    split="test",
    cache_dir="~/.maite_cache/torchvision/datasets"
)

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [12]:
dataset._dataset.save_to_disk("/dioptra/data/cifar10")

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [18]:
dataset = load_dataset(
     provider="torchvision",
     dataset_name="CIFAR10",
     task="image-classification",
     split="test",
     root="~/data",
     download=True
)


100%|████████████████████████████████████████████████████████████████| 170498071/170498071 [00:21<00:00, 8061247.80it/s]


Extracting /home/jtsexton/data/cifar-10-python.tar.gz to /home/jtsexton/data
Files already downloaded and verified


In [3]:
def create_image_dataset(
    data_dir: str,
    image_size,
    seed: int,
    validation_split = 0.2,
    batch_size: int = 32,
    label_mode: str = "categorical",
):
    """Creates an image dataset from a directory, assuming the
    subdirectories of the directory correspond to the classes of
    the data.

    Args:
        data_dir: A string representing the directory the class
            directories are located in.

        image_size:  The size in pixels of each image in the dataset.

        seed: Random seed for shuffling and transformations.

        batch_size: Size of the batches of data.

        validation_split: A float value representing the split between
            training and validation data, if desired.

        label_mode: One of 'int', 'categorical', or 'binary' depending on how the
            classes are organized.

    Returns:
        One or two DataLoader object(s) which can be used to iterate over
        images in the dataset. This will return two DataLoaders if
        validation_split is set, otherwise it will return one.
    """
    color_mode: str = "color" if image_size[2] == 3 else "grayscale"
    target_size: Tuple[int, int] = image_size[:2]

    if color_mode == "grayscale":
        transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Grayscale(),
            ]
        )
    else:
        transform = transforms.Compose([transforms.ToTensor()])

    dataset = ImageFolder(root=data_dir, transform=transform)
    
    if validation_split != None:
        train_size = (int)(validation_split * len(dataset))
        val_size = len(dataset) - (int)(validation_split * len(dataset))

        train, val = random_split(dataset, [train_size, val_size])

        train_gen = DataLoader(train, batch_size=batch_size, shuffle=True)
        val_gen = DataLoader(val, batch_size=batch_size, shuffle=True)
        return (train_gen, val_gen, list(dataset.class_to_idx.keys()))
    else:
        data_generator = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return (data_generator, None, list(dataset.class_to_idx.keys()))


In [4]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset, random_split

In [5]:
ds_train, ds_val, classes = create_image_dataset('/dioptra/data/Mnist/training',[28,28,1],0)

In [6]:
from maite._internals.interop.torchvision.datasets import TorchVisionDataset
ds_train.dataset.classes = classes
dataset = TorchVisionDataset(ds_train.dataset)

In [7]:
type(dataset.dataset)

AttributeError: 'TorchVisionDataset' object has no attribute 'dataset'

In [10]:
from torchvision.transforms.functional import to_tensor
dataset.set_transform(
    lambda x: {
        "image": x["image"].resize(28,28,1),
        "label": x["label"]
    }
)
data = dataset[0]
print(data["image"].shape)


torch.Size([28, 28, 1])


In [72]:
models = list_models(
    provider="huggingface",
    task="image-classification",
    model_name="cifar10"
)
sorted([m.id for m in models])[:20]

NameError: name 'list_models' is not defined

In [73]:
from maite import load_model

model = load_model(
    provider="huggingface",
    model_name="aaraki/vit-base-patch16-224-in21k-finetuned-cifar10",
    task="image-classification"
)

/home/jtsexton/.local/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [55]:
data = dataset[0]
output = model(data["image"])

In [56]:
output

HuggingFaceProbs(probs=tensor([[0.0153, 0.0173, 0.0247, 0.0242, 0.0213, 0.0314, 0.8189, 0.0145, 0.0172,
         0.0153]], grad_fn=<SoftmaxBackward0>))

In [57]:
print(output.probs.argmax(dim=1).item())
print(dataset[0]["label"])


6
6


In [58]:
from maite import list_metrics
len(list_metrics(provider="torchmetrics"))

93

In [59]:
len(list_metrics(provider="torcheval"))

57

In [60]:
from maite import load_metric
metric = load_metric(
    provider="torchmetrics",
    metric_name="Accuracy",
    task="multiclass",
    num_classes=10
)

In [61]:
metric

MulticlassAccuracy()

In [62]:
from maite import evaluate

evaluator = evaluate(task="image-classification")

In [63]:
dataset_subset = [dataset[i] for i in range(256)]

In [ ]:
dataset_subset

In [65]:
output = evaluator(
    model,
    dataset_subset,
    metric=dict(accuracy=metric),
    batch_size=32,
)


  0%|          | 0/8 [00:00<?, ?it/s]

In [66]:
output

{'accuracy': tensor(0.9883)}

In [29]:
from maite.interop.provider import list_providers
list_providers()

['artifact_hub']

In [13]:
import art
